In [1]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

In [2]:
with open('날개.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

23293

In [4]:
text[-20:]

'만 더 날아 보자꾸나. <1936>\n'

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True) 
tokenizer.fit_on_texts(text)

In [6]:
len(tokenizer.word_index)

813

max_id : 고유 글자 개수 (1,247개) 

dataset_size: 전체 글자 개수 (239,261개)

In [7]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [8]:
print(max_id)
print(dataset_size)

813
23293


전체 텍스트를 Encoding해서 각 글자를 ID로 나타내기 

(0부터 시작하기 위해서 1 빼기) 

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1

텍스트의 처음 90%를 훈련 세트로 사용 

이 텍스트 세트에서 한 번에 한 글자씩 반환하는 tf.data.Dataset 객체 만들기 

In [10]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

window() method를 사용해서 여러 텍스트 window로 변환 

100(모델이 보는 데이터) + 1(모델이 맞추어야 하는 데이터) 

RNN은 이 부분 문자열 길이만큼만 Back-propagation 

Truncated BackPropagation Through Time(TBPTT) 

In [11]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

모델의 실제 입력으로 사용되는 tensor 형태로 변환 

window마다 batch(window_length)를 호출 

In [12]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

One-hot encoding 

In [15]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)

(batch_size, n_steps, max_id) (batch_size, n_steps) 

In [16]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 813) (32, 100)


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(max_id, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [20]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

Epoch 1/10
655/655 [==============================] - 316s 480ms/step - loss: 4.3801 - accuracy: 0.2583
Epoch 2/10
655/655 [==============================] - 307s 469ms/step - loss: 4.3704 - accuracy: 0.2590
Epoch 3/10
655/655 [==============================] - 309s 471ms/step - loss: 4.3689 - accuracy: 0.2590
Epoch 4/10
655/655 [==============================] - 310s 474ms/step - loss: 4.3704 - accuracy: 0.2590
Epoch 5/10
655/655 [==============================] - 310s 473ms/step - loss: 4.3676 - accuracy: 0.2591
Epoch 6/10
655/655 [==============================] - 337s 515ms/step - loss: 4.3686 - accuracy: 0.2591
Epoch 7/10
655/655 [==============================] - 313s 478ms/step - loss: 4.3675 - accuracy: 0.2591
Epoch 8/10
655/655 [==============================] - 309s 472ms/step - loss: 4.3683 - accuracy: 0.2592
Epoch 9/10
655/655 [==============================] - 310s 473ms/step - loss: 4.3676 - accuracy: 0.2590
Epoch 10/10
655/655 [==============================] - 309s 472m

In [21]:
# Check its architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, None, 128)         482304    
                                                                 
 activation_3 (Activation)   (None, None, 128)         0         
                                                                 
 lstm_4 (LSTM)               (None, None, 64)          49408     
                                                                 
 activation_4 (Activation)   (None, None, 64)          0         
                                                                 
 lstm_5 (LSTM)               (None, None, 32)          12416     
                                                                 
 activation_5 (Activation)   (None, None, 32)          0         
                                                                 
 dense_1 (Dense)             (None, None, 813)        

모델 사용하기

모델에 새로운 텍스트 입력을 위한 전처리 

In [22]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [29]:
X_new = preprocess(["날개야 다시 돋"])
y_prob = model.predict(X_new)
Y_pred = y_prob.argmax(axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

' '

In [30]:
tf.random.set_seed(42)
# https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/random/categorical 
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=5).numpy()

array([[0, 1, 0, 2, 1]])

다음 글자를 온도에 따라 선택 

온도가 매우 높으면 모든 글자가 동일한 확률을 가짐 

In [31]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [33]:
tf.random.set_seed(42)

next_char("날개야 다시 돋", temperature=1)

'\n'

In [34]:
next_char("날개야 다시 돋", temperature=0.1)

' '

다음 글자를 반복적으로 얻어서 텍스트에 추가 

In [35]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [36]:
tf.random.set_seed(42)

print(complete_text("이", temperature=0.1))

이                                                  


In [37]:
print(complete_text("이", temperature=0.25))

이                                                  


In [38]:
print(complete_text("이", temperature=0.5))

이      하 는      한           내                   이  


In [39]:
print(complete_text("이", temperature=0.75))

이굴    는나      내.을이만게 . 가풍  가서 고날    불신  다는시 의    여 


In [40]:
print(complete_text("이", temperature=1))

이티있나를 았다 틀이 됐가겠 는적당  그   지  다  번도화느 까내하그  는를는 하  을어


In [41]:
print(complete_text("이", temperature=2))

이않면은란늘뒤인목배일역아짐의르이히동
빈나천없증로돋그한규돌니기뿐고벙는좋히쌉자후다면구결소가줄망.


In [42]:
print(complete_text("나", temperature=0.2))

나                                                  


In [43]:
print(complete_text("나", temperature=0.25))

나                                                  


In [44]:
print(complete_text("나", temperature=0.5))

나    그                                 하    이      


In [45]:
print(complete_text("나", temperature=0.75))

나내 달은
 까도는많다  ,을 지무 아    터 는내    는.   는 사 도    내   


In [46]:
print(complete_text("나", temperature=1))

나이,리잠  구 고 가낀다  .리달에을불좀심개다늘처?것줄 않좀아있,면소것 는 뒹다생지 람에의


In [47]:
print(complete_text("나", temperature=2))

나뒤방순로돈재이완규 없렸제 일쓰환났재심엇송서긴낙눕고 3않각후똑늘군건
뚫서쾌러결람-생셔법겼피망


In [48]:
print(complete_text("병마", temperature=0.25))

병마                                                  
